In [1]:
!pip install yfinance

# ***1. ดึงข้อมูลแบบ Trading Days (วันเปิดตลาดเท่านั้น)***

In [8]:
# นำเข้าไลบรารีที่จำเป็น
import yfinance as yf               # ไลบรารีสำหรับดึงข้อมูลหุ้นจาก Yahoo Finance
import pandas as pd                 # ไลบรารีสำหรับจัดการข้อมูลแบบตาราง (DataFrame)
from datetime import datetime, timedelta  # โมดูลจัดการวันที่และช่วงเวลา

# รายชื่อหุ้นที่ต้องการดาวน์โหลดข้อมูล
tickers = [
     "GOOGL"]  # list ของ Ticker Symbol หุ้นหลายตัว

# กำหนดช่วงเวลา 10 ปี
end_date = datetime.today()                        # วันสิ้นสุด = วันนี้
start_date = end_date - timedelta(days=5*365)     # วันเริ่มต้น = 10 ปีก่อน (ประมาณ 3650 วัน)

# สร้าง list ว่างเพื่อเก็บข้อมูลหุ้นแต่ละตัว
all_data_list = []

# วนลูปดาวน์โหลดข้อมูลหุ้นแต่ละตัว
for ticker in tickers:
    print(f"📥 Downloading {ticker} ...")  # แสดงข้อความว่ากำลังดาวน์โหลดหุ้นตัวไหน

    # ดาวน์โหลดข้อมูลหุ้นจาก Yahoo Finance ในช่วงที่กำหนด
    data = yf.download(
        ticker,
        start=start_date.strftime('%Y-%m-%d'),  # แปลงวันที่เป็น string 'YYYY-MM-DD'
        end=end_date.strftime('%Y-%m-%d')
    )

    # ตรวจสอบว่ามีข้อมูลหรือไม่
    if data.empty:
        print(f"❌ No data for {ticker}")  # ถ้าไม่มีข้อมูล แสดงข้อความ
        continue                             # ข้ามไปหุ้นตัวถัดไป

    # รีเซ็ต index ของ DataFrame (จาก Date index → column ปกติ)
    data.reset_index(inplace=True)

    # ถ้ามี MultiIndex columns (มักเกิดจาก group_by='ticker') ให้แปลงเป็น columns ปกติ
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [
            ' '.join(col).strip() if isinstance(col, tuple) else col
            for col in data.columns.values
        ]

    # ฟังก์ชันทำความสะอาดชื่อคอลัมน์ เช่น 'Close GOOGL' → 'Close'
    def clean_columns(cols):
        cleaned = []
        for col in cols:
            if 'Date' in col:               # ถ้าเป็นคอลัมน์วันที่
                cleaned.append('Date')
            else:
                cleaned.append(col.split()[0])  # เอาเฉพาะคำแรก เช่น 'Close', 'Open'
        return cleaned

    # ใช้ฟังก์ชัน clean_columns กับ DataFrame
    data.columns = clean_columns(data.columns)

    # เพิ่มคอลัมน์ Symbol เพื่อเก็บชื่อหุ้น
    data['Symbol'] = ticker.upper()  # ใช้ตัวพิมพ์ใหญ่ให้ uniform

    # กรองเฉพาะคอลัมน์ที่ต้องการ
    wanted_cols = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Symbol']
    data = data[[col for col in wanted_cols if col in data.columns]]  # เฉพาะคอลัมน์ที่มีจริง

    # เพิ่ม DataFrame ของหุ้นตัวนี้ลงใน list
    all_data_list.append(data)

# รวมทุก DataFrame เป็น flat table เดียว
flat_df = pd.concat(all_data_list, ignore_index=True)

# แปลงคอลัมน์ Date เป็น datetime object
flat_df['Date'] = pd.to_datetime(flat_df['Date'])

# ทำให้ Symbol เป็น string แบบ uniform (ตัวพิมพ์ใหญ่)
flat_df['Symbol'] = flat_df['Symbol'].astype(str).str.upper()

# เรียงลำดับก่อน → Symbol, จากนั้น → Date
flat_df = flat_df.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)

# ตรวจสอบชนิดข้อมูลแต่ละคอลัมน์
print(flat_df.dtypes)

# แสดงตัวอย่าง 10 แถวแรก
print(flat_df.head(10))

📥 Downloading GOOGL ...


/tmp/ipython-input-791219495.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed

Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
Symbol            object
dtype: object
        Date      Close       High        Low       Open    Volume Symbol
0 2020-08-26  81.716423  82.146848  79.571791  79.836204  52188000  GOOGL
1 2020-08-27  80.940575  81.908267  80.457965  81.839678  31350000  GOOGL
2 2020-08-28  81.482819  81.578247  80.795944  80.987794  22418000  GOOGL
3 2020-08-31  80.990776  81.734811  80.782029  81.688587  26422000  GOOGL
4 2020-09-01  82.260651  82.466417  80.990770  81.121487  22652000  GOOGL
5 2020-09-02  85.357582  85.790487  82.514632  82.903299  49522000  GOOGL
6 2020-09-03  80.989784  84.493271  79.906278  84.469412  63726000  GOOGL
7 2020-09-04  78.589188  81.262153  76.440074  79.970400  55850000  GOOGL
8 2020-09-08  75.725845  77.313818  75.372958  75.795428  54032000  GOOGL
9 2020-09-09  76.900314  77.471383  75.803885  76.983313  39242000  GOO

In [9]:
flat_df

,Date,Close,High,Low,Open,Volume,Symbol
0,2020-08-26,81.716423,82.146848,79.571791,79.836204,52188000,GOOGL
1,2020-08-27,80.940575,81.908267,80.457965,81.839678,31350000,GOOGL
2,2020-08-28,81.482819,81.578247,80.795944,80.987794,22418000,GOOGL
3,2020-08-31,80.990776,81.734811,80.782029,81.688587,26422000,GOOGL
4,2020-09-01,82.260651,82.466417,80.990770,81.121487,22652000,GOOGL
...,...,...,...,...,...,...,...
1249,2025-08-18,203.500000,205.270004,202.490005,204.199997,18526600,GOOGL
1250,2025-08-19,201.570007,203.440002,199.960007,203.029999,24240200,GOOGL
1251,2025-08-20,199.320007,201.279999,196.600006,200.729996,28955500,GOOGL
1252,2025-08-21,199.750000,202.479996,199.429993,199.750000,19774600,GOOGL


In [10]:
flat_df.dtypes

,0
Date,datetime64[ns]
Close,float64
High,float64
Low,float64
Open,float64
Volume,int64
Symbol,object


In [11]:
!pip install pandas_market_calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 10.1 MB/s eta 0:00:00


# ***2. สร้างข้อมูลแบบ Calendar Days (รวมวันหยุดและเสาร์-อาทิตย์)***


In [12]:
# นำเข้าไลบรารีที่จำเป็น
import yfinance as yf               # ไลบรารีสำหรับดึงข้อมูลหุ้นจาก Yahoo Finance
import pandas as pd                 # ไลบรารีสำหรับจัดการข้อมูลแบบตาราง (DataFrame)
from datetime import datetime, timedelta  # โมดูลจัดการวันที่และช่วงเวลา

# รายชื่อหุ้นที่ต้องการดาวน์โหลดข้อมูล
tickers = [
     "GOOGL"]  # list ของ Ticker Symbol หุ้นหลายตัว

# กำหนดช่วงเวลา 10 ปี
end_date = datetime.today()                        # วันสิ้นสุด = วันนี้
start_date = end_date - timedelta(days=5*365)     # วันเริ่มต้น = 10 ปีก่อน (ประมาณ 3650 วัน)

# สร้าง list ว่างเพื่อเก็บข้อมูลหุ้นแต่ละตัว
all_data_list = []

# วนลูปดาวน์โหลดข้อมูลหุ้นแต่ละตัว
for ticker in tickers:
    print(f"📥 Downloading {ticker} ...")  # แสดงข้อความว่ากำลังดาวน์โหลดหุ้นตัวไหน

    # ดาวน์โหลดข้อมูลหุ้นจาก Yahoo Finance ในช่วงที่กำหนด
    data = yf.download(
        ticker,
        start=start_date.strftime('%Y-%m-%d'),  # แปลงวันที่เป็น string 'YYYY-MM-DD'
        end=end_date.strftime('%Y-%m-%d')
    )

    # ตรวจสอบว่ามีข้อมูลหรือไม่
    if data.empty:
        print(f"❌ No data for {ticker}")  # ถ้าไม่มีข้อมูล แสดงข้อความ
        continue                             # ข้ามไปหุ้นตัวถัดไป

    # รีเซ็ต index ของ DataFrame (จาก Date index → column ปกติ)
    data.reset_index(inplace=True)

    # ถ้ามี MultiIndex columns (มักเกิดจาก group_by='ticker') ให้แปลงเป็น columns ปกติ
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [
            ' '.join(col).strip() if isinstance(col, tuple) else col
            for col in data.columns.values
        ]

    # ฟังก์ชันทำความสะอาดชื่อคอลัมน์ เช่น 'Close GOOGL' → 'Close'
    def clean_columns(cols):
        cleaned = []
        for col in cols:
            if 'Date' in col:               # ถ้าเป็นคอลัมน์วันที่
                cleaned.append('Date')
            else:
                cleaned.append(col.split()[0])  # เอาเฉพาะคำแรก เช่น 'Close', 'Open'
        return cleaned

    # ใช้ฟังก์ชัน clean_columns กับ DataFrame
    data.columns = clean_columns(data.columns)

    # เพิ่มคอลัมน์ Symbol เพื่อเก็บชื่อหุ้น
    data['Symbol'] = ticker.upper()  # ใช้ตัวพิมพ์ใหญ่ให้ uniform

    # กรองเฉพาะคอลัมน์ที่ต้องการ
    wanted_cols = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Symbol']
    data = data[[col for col in wanted_cols if col in data.columns]]  # เฉพาะคอลัมน์ที่มีจริง

    # เพิ่ม DataFrame ของหุ้นตัวนี้ลงใน list
    all_data_list.append(data)

# รวมทุก DataFrame เป็น flat table เดียว
flat_df = pd.concat(all_data_list, ignore_index=True)

# แปลงคอลัมน์ Date เป็น datetime object
flat_df['Date'] = pd.to_datetime(flat_df['Date'])

# ทำให้ Symbol เป็น string แบบ uniform (ตัวพิมพ์ใหญ่)
flat_df['Symbol'] = flat_df['Symbol'].astype(str).str.upper()

# เรียงลำดับก่อน → Symbol, จากนั้น → Date
flat_df = flat_df.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)

import pandas as pd
import pandas_market_calendars as mcal

# 1. สร้าง continuous date range
all_dates = pd.DataFrame({'Date': pd.date_range(flat_df['Date'].min(), flat_df['Date'].max())})

# 2. ขยาย Symbol list ให้ครบทุกวัน
symbols = flat_df['Symbol'].unique()
expanded_list = []
for sym in symbols:
    temp = all_dates.copy()
    temp['Symbol'] = sym
    expanded_list.append(temp)
all_dates_symbols = pd.concat(expanded_list, ignore_index=True)

# 3. merge กับ flat_df ของราคาหุ้น
full_df = all_dates_symbols.merge(flat_df, on=['Date','Symbol'], how='left')


/tmp/ipython-input-3886553684.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed

📥 Downloading GOOGL ...


In [13]:
full_df

,Date,Symbol,Close,High,Low,Open,Volume
0,2020-08-26,GOOGL,81.716423,82.146848,79.571791,79.836204,52188000.0
1,2020-08-27,GOOGL,80.940575,81.908267,80.457965,81.839678,31350000.0
2,2020-08-28,GOOGL,81.482819,81.578247,80.795944,80.987794,22418000.0
3,2020-08-29,GOOGL,NaN,NaN,NaN,NaN,NaN
4,2020-08-30,GOOGL,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1818,2025-08-18,GOOGL,203.500000,205.270004,202.490005,204.199997,18526600.0
1819,2025-08-19,GOOGL,201.570007,203.440002,199.960007,203.029999,24240200.0
1820,2025-08-20,GOOGL,199.320007,201.279999,196.600006,200.729996,28955500.0
1821,2025-08-21,GOOGL,199.750000,202.479996,199.429993,199.750000,19774600.0
